In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [18]:
Jdata = pd.read_csv("cleandata.csv")
Jdata


,Unnamed: 0,position,company,description,location
0,3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,"Atlanta, GA 30303"
1,74,Data Analyst,The Home Depot,POSITION PURPOSE:\nA Data Analyst Supply Chain...,"Atlanta, GA"
2,251,Data Analyst,The Home Depot,POSITION PURPOSE:\nA Sr Data Analyst Supply Ch...,"Atlanta, GA"
3,298,Data Analyst,Applied Systems Inc.,Job Description\nMake a Difference Every Day w...,"Austin, TX"
4,331,Data Analyst,Pacific Investment Management Company LLC,PIMCO is a global investment solutions provide...,"Austin, TX"
...,...,...,...,...,...
1673,6207,Research Analyst,JLL,"Perform complex financial analyses, including ...","Menlo Park, CA 94025"
1674,6255,Research Analyst,SEIU United Service Workers West,We are looking for someone who is self-startin...,"Alameda, CA"
1675,6351,Research Analyst,Nielsen,Research Analyst - 23816\nAnalytics - USA San ...,"San Francisco, CA"
1676,6503,Research Analyst,CBRE,JOB SUMMARY\n\nProvide research and analysis s...,"San Francisco, CA"


In [19]:
#group by
grouped = Jdata.groupby('location').count()
#sort_values(ascending=False)
grouped.head(250)

,Unnamed: 0,position,company,description
location,,,,
"Alameda, CA",3,3,3,3
"Atlanta, GA",31,31,31,31
"Atlanta, GA 30301",2,2,2,2
"Atlanta, GA 30303",2,2,2,2
"Atlanta, GA 30308",1,1,1,1
...,...,...,...,...
"Washington, DC 20230",2,2,2,2
"Washington, DC 20318",1,1,1,1
"Washington, DC 20585",1,1,1,1


In [20]:
grouped_citystate = Jdata.groupby('location')['position'].count().sort_values(ascending=False).reset_index()
grouped_citystate.head(300)


,location,position
0,"New York, NY",238
1,"San Francisco, CA",182
2,"Seattle, WA",105
3,"Washington, DC",89
4,"Boston, MA",88
...,...,...
222,"Oakland, CA 94601",1
223,"Oakland, CA 94607",1
224,"Palisades, NY",1
225,"Port Washington, NY 11050",1


In [21]:
grouped_citystate['lat'] = 999
grouped_citystate['lng']= 999
grouped_citystate

,location,position,lat,lng
0,"New York, NY",238,999,999
1,"San Francisco, CA",182,999,999
2,"Seattle, WA",105,999,999
3,"Washington, DC",89,999,999
4,"Boston, MA",88,999,999
...,...,...,...,...
222,"Oakland, CA 94601",1,999,999
223,"Oakland, CA 94607",1,999,999
224,"Palisades, NY",1,999,999
225,"Port Washington, NY 11050",1,999,999


In [26]:
for irow in np.arange(len(grouped_citystate)):
    print(irow)
    target_city = grouped_citystate.location[irow]
    # Build the endpoint URL
    target_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={g_key}'
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
#     print(json.dumps(geo_data, indent=4, sort_keys=True))
    # Extract latitude and longitude
    try:
        grouped_citystate.loc[irow, 'lat'] = geo_data["results"][0]["geometry"]["location"]["lat"]
        grouped_citystate.loc[irow, 'lng'] = geo_data["results"][0]["geometry"]["location"]["lng"]
    except:
        grouped_citystate.loc[irow, 'lat'] = '999'
        grouped_citystate.loc[irow, 'lng'] = '999'

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226


In [28]:

grouped_citystate

,location,position,lat,lng
0,"New York, NY",238,40.712775,-74.005973
1,"San Francisco, CA",182,37.774929,-122.419415
2,"Seattle, WA",105,47.606209,-122.332071
3,"Washington, DC",89,38.907192,-77.036871
4,"Boston, MA",88,42.360082,-71.058880
...,...,...,...,...
222,"Oakland, CA 94601",1,37.772927,-122.214332
223,"Oakland, CA 94607",1,37.813468,-122.307917
224,"Palisades, NY",1,41.011308,-73.913293
225,"Port Washington, NY 11050",1,40.847001,-73.694823


In [30]:
locations = grouped_citystate[["lat", "lng"]].astype(float)

grouped_citystate.position


0      238
1      182
2      105
3       89
4       88
      ... 
222      1
223      1
224      1
225      1
226      1
Name: position, Length: 227, dtype: int64

In [32]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=grouped_citystate.position, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))